In [ ]:
from pymongo  import MongoClient
from wordcloud import WordCloud
import pandas as pd
import re
import collections
from pathlib import Path
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import string
import scipy.stats as st
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn import decomposition, naive_bayes, preprocessing, model_selection, metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection
from tqdm.notebook import tqdm
#! pip install scikit-surprise
from surprise import NormalPredictor, BaselineOnly, SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


# Chargement des données AVIS

In [ ]:
client = MongoClient(host="localhost", port=27017)
db = client["PLDAC"]
collection = db["avis"]

In [ ]:
df_avis = pd.DataFrame(list(collection.find())).loc[:,["author","title","note"]]
df_avis.head(5)

In [ ]:
df_avis['author'].value_counts().describe()

# Suppression des autheurs qui ont noté moins de 5 jeux

In [ ]:
print(f"Taille du df avant {len(df_avis)}")
# Taille du df avant 246524

authors = df_avis['author'].value_counts()
authors = authors[authors >= 10].index.to_list()

df_avis_k = df_avis[df_avis['author'].isin(authors)]
print(f"Taille du df après {len(df_avis_k)}")
# Taille du df après 5925

# Test/Train split


In [ ]:
X_train, X_test = model_selection.train_test_split(df_avis_k, test_size=0.2, random_state=0)
df_avis.head()

## Basic Recsys **mean** baselines implementations


In [ ]:
MEAN = X_train["note"].mean()
USER_MEANS = X_train.groupby("author")["note"].mean()
ITEM_MEANS = X_train.groupby("title")["note"].mean()


def mean_rating_pred(user_item):
    user = user_item["author"]
    item = user_item["title"]
    
    return MEAN

def user_mean_rating_pred(user_item):
    user = user_item["author"]
    item = user_item["title"]
    
    return USER_MEANS.get(user,default=MEAN)

def item_mean_rating_pred(user_item):
    user = user_item["author"]
    item = user_item["title"]
    
    return ITEM_MEANS.get(item,default=MEAN)


In [ ]:
#We create the rating prediction columns
X_test["mean_prediction"] = X_test[["author","title"]].apply(mean_rating_pred,axis=1)
X_test["muser_prediction"] = X_test[["author","title"]].apply(user_mean_rating_pred,axis=1) 
X_test["mitem_prediction"] = X_test[["author","title"]].apply(item_mean_rating_pred,axis=1) 

X_test.head(5)

In [ ]:
from math import sqrt

def mae(predictions,truth):
    return ((predictions-truth).abs()).mean()

def mse(predictions,truth):
    return ((predictions - truth)**2).mean()

def rmse(predictions,truth):
    return sqrt(mse(predictions,truth))


def all_metrics(predictions,truth):
    return [f(predictions,truth) for f in [mae,mse,rmse]]

In [ ]:
metrics = ["mae","mse","rmse"]
results = pd.DataFrame()

results["metrics"] = metrics
results["mean_prediction"] = all_metrics(X_test["mean_prediction"],X_test["note"])
results["muser_prediction"] = all_metrics(X_test["muser_prediction"],X_test["note"])
results["mitem_prediction"] = all_metrics(X_test["mitem_prediction"],X_test["note"])
results = results.set_index("metrics")

print(results)
print("")
print('---Best Models / Metrics: ---')
results.idxmin(1)

In [ ]:
## Uncomment this to install required packages if needed (and restart kernel !)
#! pip install --upgrade scikit-surprise

In [ ]:
from surprise import NormalPredictor, BaselineOnly, SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(X_test[['author', 'title', 'note']], Reader(rating_scale=(1, 10)))

In [ ]:
BaselineModel = BaselineOnly()
BaselineModel.fit(data.build_full_trainset())

In [ ]:
def opt_bl_rating_pred(user_item):
    user = user_item["author"]
    item = user_item["title"]
    
    prediction = BaselineModel.predict(user,item)
    
    return prediction.est

X_test["opt_bl_prediction"] = X_test[["author","title"]].apply(opt_bl_rating_pred,axis=1) 

X_test

In [ ]:

SVDmodel = SVD()
SVDmodel.fit(data.build_full_trainset())